In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [14]:
def get_data_loader(training = True):
    cust_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    data_set = datasets.FashionMNIST('./data', train=training,
                                     download=True, transform=cust_transform)
    loader = torch.utils.data.DataLoader(data_set, batch_size=64)
    return loader

In [15]:
def build_model():
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28,128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )
    return model


In [26]:
def train_model(model, train_loader, criterion, T):
    opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(T):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            opt.zero_grad()

            outputs = model(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            opt.step()

            running_loss += loss.item()
        accuracy = 100 * correct / total
        avg_loss = running_loss / len(train_loader)
        print('Train Epoch: {} Accuracy: {}/{}({:.2f}%) Loss: {:.3f}'.format(
            epoch, correct, total, accuracy, avg_loss))

In [25]:
def evaluate_model(model, test_loader, criterion, show_loss = True):
    model.eval()
    running_loss = 0.0
    accurate = 0
    total = 0
    with torch.no_grad():
        for input, label in test_loader:
            output = model(input)
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            accurate += (predicted == label).sum().item()
            loss = criterion(output, label)
            running_loss += loss.item()
    if show_loss:
        print("Average loss: {:.4f}".format(running_loss/len(test_loader.dataset)))    
    print("Accuracy: {:.2f}%".format(100*accurate/total))

In [18]:
def predict_label(model, test_images, index):
    class_names = ['T-shirt/top','Trouser','Pullover','Dress',
                   'Coat','Sandal','Shirt','Sneaker','Bag',
                   'Ankle Boot']
    logits = model(test_images[index])
    prob = F.softmax(logits, dim=1)
    prob_list = prob.tolist()
    flat_list = [item for sublist in prob_list for item in sublist]
    map = {class_names[i]: flat_list[i]*100 for i in range(len(class_names))}
    sorted_map = sorted(map.items(), key=lambda x:x[1], reverse=True)
    for i in range(3):
        print(f'{sorted_map[i][0]}: {sorted_map[i][1]:.2f}%')


In [19]:
train_loader = get_data_loader()
print(type(train_loader))
print(train_loader.dataset)

test_loader = get_data_loader(training=False)
print(test_loader.dataset)

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [20]:
model = build_model()
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=10, bias=True)
)


In [27]:
criterion = nn.CrossEntropyLoss()
train_model(model, train_loader, criterion, 5)

Train Epoch: 0 Accuracy: 51839/60000(86.40%) Loss: 0.386
Train Epoch: 1 Accuracy: 52083/60000(86.81%) Loss: 0.371
Train Epoch: 2 Accuracy: 52301/60000(87.17%) Loss: 0.358
Train Epoch: 3 Accuracy: 52518/60000(87.53%) Loss: 0.347
Train Epoch: 4 Accuracy: 52738/60000(87.90%) Loss: 0.337


In [22]:
evaluate_model(model, test_loader, criterion, show_loss=True)

Average loss: 0.4301
Accuracy: 84.48%


In [23]:
evaluate_model(model, test_loader, criterion, show_loss=False)

Accuracy: 84.48%


In [24]:
test_images = next(iter(test_loader))[0]
predict_label(model, test_images, 1)

Pullover: 88.71%
Shirt: 10.30%
Coat: 0.86%
